![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

________________________________________________________________________________________________________

In [49]:
# Start your code here!
import pandas as pd

In [50]:
# Load data
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [51]:
ds_jobs.memory_usage()

Index                        132
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [52]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [53]:
# Change int64 data type into int32
ds_jobs_clean = pd.DataFrame()
ds_jobs_clean['student_id'] = ds_jobs['student_id'].astype('int32')
ds_jobs_clean['training_hours'] = ds_jobs['training_hours'].astype('int32')
ds_jobs_clean['job_change'] = ds_jobs['job_change'].astype('int32')

# Change float64 data type into float16
ds_jobs_clean['city_development_index'] = ds_jobs['city_development_index'].astype('float16')

In [55]:
# Print unique value for each non-numerical column to determine whether it's nominal/ordinal data

for column in ds_jobs.columns:
    if not pd.api.types.is_numeric_dtype(ds_jobs[column]):
        unique_values = ds_jobs[column].unique()
        print(f"Unique values in {column}: {unique_values} \n")

Unique values in city: ['city_103' 'city_40' 'city_21' 'city_115' 'city_162' 'city_176'
 'city_160' 'city_46' 'city_61' 'city_114' 'city_13' 'city_159' 'city_102'
 'city_67' 'city_100' 'city_16' 'city_71' 'city_104' 'city_64' 'city_101'
 'city_83' 'city_105' 'city_73' 'city_75' 'city_41' 'city_11' 'city_93'
 'city_90' 'city_36' 'city_20' 'city_57' 'city_152' 'city_19' 'city_65'
 'city_74' 'city_173' 'city_136' 'city_98' 'city_97' 'city_50' 'city_138'
 'city_82' 'city_157' 'city_89' 'city_150' 'city_70' 'city_175' 'city_94'
 'city_28' 'city_59' 'city_165' 'city_145' 'city_142' 'city_26' 'city_12'
 'city_37' 'city_43' 'city_116' 'city_23' 'city_99' 'city_149' 'city_10'
 'city_45' 'city_80' 'city_128' 'city_158' 'city_123' 'city_7' 'city_72'
 'city_106' 'city_143' 'city_78' 'city_109' 'city_24' 'city_134' 'city_48'
 'city_144' 'city_91' 'city_146' 'city_133' 'city_126' 'city_118' 'city_9'
 'city_167' 'city_27' 'city_84' 'city_54' 'city_39' 'city_79' 'city_76'
 'city_77' 'city_81' 'city_13

In [63]:
# Convert into category data type
ds_jobs_clean["city"] = ds_jobs["city"].astype("category")
ds_jobs_clean["gender"] = ds_jobs["city"].astype("category")
ds_jobs_clean["relevant_experience"] = ds_jobs["relevant_experience"].astype("category")
ds_jobs_clean["enrolled_university"] = ds_jobs["enrolled_university"].astype("category")
ds_jobs_clean["education_level"] = ds_jobs["education_level"].astype("category")
ds_jobs_clean["major_discipline"] = ds_jobs["major_discipline"].astype("category")
ds_jobs_clean["experience"] = ds_jobs["experience"].astype("category")
ds_jobs_clean["company_size"] = ds_jobs["company_size"].astype("category")
ds_jobs_clean["company_type"] = ds_jobs["company_type"].astype("category")
ds_jobs_clean["last_new_job"] = ds_jobs["last_new_job"].astype("category")

In [66]:
# Convert into ordinal data type
ds_jobs_clean["education_level"] = ds_jobs_clean["education_level"].cat.set_categories(
    new_categories=["Primary School", "High School", "Graduate", "Masters", "Phd"])

ds_jobs_clean["experience"] = ds_jobs_clean["education_level"].cat.set_categories(
    new_categories=["<1", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10",
                    "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", ">20"])

ds_jobs_clean["company_size"] = ds_jobs_clean["company_size"].cat.set_categories(
    new_categories=["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"])

ds_jobs_clean["last_new_job"] = ds_jobs_clean["last_new_job"].cat.set_categories(
    new_categories=["never", "1", "2", "3", "4", ">4"])